In [1]:
%config Completer.use_jedi = False

In [2]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy
from mpl_toolkits.axes_grid1 import make_axes_locatable
import xarray as xr
import matplotlib.animation as animation
from IPython import display
import tqdm
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.interpolate import cross_section
from metpy.units import units
import pickle
from regrid import regrid
from netCDF4 import Dataset
import numpy as np
import datetime
from calendar import monthrange
import mask
from cartoplot import cartoplot

/home/robbie/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/robbie/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


# Get generic data

In [4]:
e0 = datetime.date(1900,1,1)

with Dataset('/media/robbie/Brokendrive/E5/ERA_nh_1973_12.nc') as ERA5_data:
    lons = np.array(ERA5_data['longitude'])
    lats = np.array(ERA5_data['latitude'])
    
ERA5_lon_grid = np.array([np.array(lons), ] * 121)

ERA5_lat_grid = np.array([np.array(lats), ] * 1440).T

area_weighting = np.cos(np.radians(ERA5_lat_grid))

In [5]:
mymask, elons, elats = mask.get('mask'), mask.get('lon'), mask.get('lat')

ERA_mask = regrid(mymask,elons,elats,ERA5_lon_grid,ERA5_lat_grid)

ones = np.ones(ERA5_lon_grid.shape)

m = np.ma.masked_where(np.isin(ERA_mask,[8,
                                            9,
                                            10,11,12,13,15],invert=True), ones)
m = np.ma.filled(m, 0)

area_weighting_m = np.multiply(area_weighting, m)

In [6]:
mon_weight_m = np.swapaxes(np.repeat(area_weighting_m[:,:,np.newaxis],31*8,axis=2).T,1,2)
mon_weight = np.swapaxes(np.repeat(area_weighting[:,:,np.newaxis],31*8,axis=2).T,1,2)

In [7]:
data_dir = '/home/robbie/Dropbox/beaufort_sea/monthly_mean_slp/'

# Process annual ERA5 files

## This code makes a dict for each year of mean SLP's above 60 degrees

In [9]:
years = tqdm.trange(1979,2022)

years = [2021]

for year in years:
    
    times_daily = {}
    mean_slps_daily = {}
    mean_slps_daily_m = {}
    mean_slp_pattern = {}
#     max_slps_daily = {}
    
#     print(year)
    with Dataset(f'/media/robbie/TOSHIBA EXT/E5/{year}.nc') as ERA5_data:

        d=0
        
        months_to_analyse = [12,1,2] if year < 2021 else [1,2]
        
        for month in np.arange(1,13):
            
            days_in_month = monthrange(year,month)[1]            
            
            if month in months_to_analyse:
            
#                 if year != 2020:

                slp = np.array(ERA5_data['msl'][d*8:(d+days_in_month)*8])


#                 elif (year == 2020) & (month < 12):

#                     slp = np.array(ERA5_data['msl'][d*8:(d+days_in_month)*8,0])

#                 else:            

#                     slp = np.array(ERA5_data['msl'][d*8:(d+days_in_month)*8,1])

                # slp is now a 31x1440x140 np array

                m_weights = mon_weight[:slp.shape[0]]
                
                m_weights_mask = mon_weight_m[:slp.shape[0]]

                slp_mean = np.average(slp,
                                      weights=m_weights,
                                      axis=(1,2))
                
                slp_mean_mask = np.average(slp,
                                      weights=m_weights_mask,
                                      axis=(1,2))

#                 slp_max = np.max(slp)

                t = np.array(ERA5_data['time'][d*8:(d+days_in_month)*8])

                times_daily[month]=t
                mean_slps_daily[month]=slp_mean
                mean_slps_daily_m[month]=slp_mean_mask
                mean_slp_pattern[month] = np.nanmean(slp,axis=0)

            d+=days_in_month
            
    
    mean_slp_dict = {'mean_slps':mean_slps_daily,
                     'mean_slp_pattern':mean_slp_pattern,
                     'mean_slps_m':mean_slps_daily_m,
                     'times':times_daily}
    
    pickle.dump(mean_slp_dict, open(f'{data_dir}{year}_mean_slp.p','wb'))


  0%|          | 0/43 [00:00<?, ?it/s]